## Week 5 - Sequence labelling

In [1]:
!pip install fasttext
!pip install pytorch-crf
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-linux_x86_64.whl size=4697318 sha256=c6e96ddb82a5c35c92a5bb11ff6747a5cfcb9b7ab811410073b8a3b347a06216
  Stored in directory: /home/knud/.cache/pip/wheels/93/61/2a/c54711a91c418ba06ba195b1d78ff24fcaad8592f2a694ac94
Successfully built fasttext


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import io
from math import log
from numpy import array
from numpy import argmax
import pandas as pd
import torch
import random
from math import log
from numpy import array
from numpy import argmax
import numpy as np
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import Adam
# from torchcrf import CRF
from torch.optim.lr_scheduler import ExponentialLR, CyclicLR
from typing import List, Tuple, AnyStr
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt
from copy import deepcopy
from datasets import load_dataset, load_metric
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import heapq
import spacy

2022-10-14 11:59:51.729931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-14 11:59:51.908614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-14 11:59:51.908641: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-14 11:59:51.946086: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-14 11:59:52.879084: W tensorflow/stream_executor/platform/de

In [3]:
import sys
sys.path.append("../../../../stat-nlp-book/")

import statnlpbook.util as util
import statnlpbook.sequence as seq
from statnlpbook.gmb import load_gmb_dataset

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # For atomic operations there is currently 
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    random.seed(seed)
    np.random.seed(seed)

enforce_reproducibility()

In [10]:
datasets = load_dataset("copenlu/answerable_tydiqa")
# datasets

Using custom data configuration copenlu--nlp_course_tydiqa-cceecfb5416d988a
Found cached dataset parquet (/home/gadelampe/.cache/huggingface/datasets/copenlu___parquet/copenlu--nlp_course_tydiqa-cceecfb5416d988a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

#### (a)

In [23]:
dfv = datasets.data['validation'].to_pandas()
dft = datasets.data['train'].to_pandas()

dfv = dfv.loc[dfv['language'].isin(['english', 'japanese','finnish'])].reset_index(drop=True)
dft = dft.loc[dft['language'].isin(['english', 'japanese','finnish'])].reset_index(drop=True)

dfv_eng = dfv.loc[dfv['language']=='english'].reset_index(drop=True)
# dfv_jap = dfv.loc[dfv['language']=='japanese'].reset_index(drop=True)
# dfv_fin = dfv.loc[dfv['language']=='finnish'].reset_index(drop=True)

dft_eng = dft.loc[dft['language']=='english'].reset_index(drop=True)
# dft_jap = dft.loc[dft['language']=='japanese'].reset_index(drop=True)
# dft_fin = dft.loc[dft['language']=='finnish'].reset_index(drop=True)

In [26]:
# import training data
dft_eng = pd.read_csv('../../data/dft_eng.csv')
dft_jap = pd.read_csv('../../data/dft_jap.csv')
dft_fin = pd.read_csv('../../data/dft_fin.csv')

# import validation data
dfv_eng = pd.read_csv('../../data/dfv_eng.csv')
dfv_jap = pd.read_csv('../../data/dfv_jap.csv')
dfv_fin = pd.read_csv('../../data/dfv_fin.csv')

#import word count
word_count = pd.read_csv('../../data/question_word_count.csv')

In [27]:
# tbwt = TreebankWordTokenizer()
# stan = StanfordTokenizer()
# word = word_tokenize()
punct = WordPunctTokenizer()


dfv_eng['question_text_tokenized'] = [punct.tokenize(i) for i in dfv_eng.question_text.values]
# dfv_jap['question_text_tokenized'] = [wakati.parse(i).split() for i in dfv_jap.question_text.values]
# dfv_fin['question_text_tokenized'] = [tbwt.tokenize(i) for i in dfv_fin.question_text.values]

dft_eng['question_text_tokenized'] = [punct.tokenize(i) for i in dft_eng.question_text.values]
# dft_jap['question_text_tokenized'] = [wakati.parse(i).split() for i in dft_jap.question_text.values]
# dft_fin['question_text_tokenized'] = [tbwt.tokenize(i) for i in dft_fin.question_text.values]

dfv_eng['document_plaintext_tokenized'] = [punct.tokenize(i) for i in dfv_eng.document_plaintext.values]
# dfv_jap['document_plaintext_tokenized'] = [wakati.parse(i).split() for i in dfv_jap.document_plaintext.values]
# dfv_fin['document_plaintext_tokenized'] = [tbwt.tokenize(i) for i in dfv_fin.document_plaintext.values]

dft_eng['document_plaintext_tokenized'] = [punct.tokenize(i) for i in dft_eng.document_plaintext.values]
# dft_jap['document_plaintext_tokenized'] = [wakati.parse(i).split() for i in dft_jap.document_plaintext.values]
# dft_fin['document_plaintext_tokenized'] = [tbwt.tokenize(i) for i in dft_fin.document_plaintext.values]

TypeError: expected string or bytes-like object

In [25]:
dft_eng.iloc[1]

question_text                   Who was the first Nobel prize winner for Liter...
document_title                              List of Nobel laureates in Literature
language                                                                  english
annotations                     {'answer_start': [610], 'answer_text': ['Sully...
document_plaintext              The Nobel Prize in Literature (Swedish: Nobelp...
document_url                    https://en.wikipedia.org/wiki/List%20of%20Nobe...
question_text_tokenized         [Who, was, the, first, Nobel, prize, winner, f...
document_plaintext_tokenized    [The, Nobel, Prize, in, Literature, (, Swedish...
Name: 1, dtype: object

In [16]:
d_tokens = dft_eng['document_plaintext_tokenized']

d_tokens

0       [Quantum, field, theory, naturally, began, wit...
1       [The, Nobel, Prize, in, Literature, (, Swedish...
2       [Dialectic, or, dialectics, (, Greek, :, διαλε...
3       [Hangul, was, personally, created, and, promul...
4       [Grasshoppers, are, plant, -, eaters, ,, with,...
                              ...                        
7384    [The, medley, relay, was, scheduled, in, the, ...
7385    [Sāmkhya, is, a, dualist, philosophical, tradi...
7386    [Mollo, was, surprised, by, the, success, of, ...
7387    [In, the, end, ,, President, Truman, made, the...
7388    [The, previous, mayor, ,, Bill, Laforet, faced...
Name: document_plaintext_tokenized, Length: 7389, dtype: object

In [31]:
tokens, pos, ents = load_gmb_dataset('../../../../stat-nlp-book/data/gmb/data/GMB_dataset_utf8.txt')

pd.DataFrame([tokens[2], pos[2]])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,They,marched,from,the,Houses,of,Parliament,to,a,rally,in,Hyde,Park,.
1,PRP,VBD,IN,DT,NNS,IN,NN,TO,DT,NN,IN,NNP,NNP,.


In [ ]:
def feat_1(x,i):
    return {
        'bias': 1.0,
        'word:' + x[i]: 1.0,
    }

train = list(zip(tokens[:-200], pos[:-200]))
dev = list(zip(tokens[-200:], pos[-200:]))

local_1 = seq.LocalSequenceLabeler(feat_1, train, class_weight='balanced')

In [32]:
tokens[1]

['Families',
 'of',
 'soldiers',
 'killed',
 'in',
 'the',
 'conflict',
 'joined',
 'the',
 'protesters',
 'who',
 'carried',
 'banners',
 'with',
 'such',
 'slogans',
 'as',
 '"',
 'Bush',
 'Number',
 'One',
 'Terrorist',
 '"',
 'and',
 '"',
 'Stop',
 'the',
 'Bombings',
 '.',
 '"']

In [36]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [44]:
test = nlp(dft_eng['document_plaintext'][1])

test

The Nobel Prize in Literature (Swedish: Nobelpriset i litteratur) is awarded annually by the Swedish Academy to authors for outstanding contributions in the field of literature. It is one of the five Nobel Prizes established by the 1895 will of Alfred Nobel, which are awarded for outstanding contributions in chemistry, physics, literature, peace, and physiology or medicine.[1] As dictated by Nobel's will, the award is administered by the Nobel Foundation and awarded by a committee that consists of five members elected by the Swedish Academy.[2] The first Nobel Prize in Literature was awarded in 1901 to Sully Prudhomme of France.[3] Each recipient receives a medal, a diploma and a monetary award prize that has varied throughout the years.[4] In 1901, Prudhomme received 150,782 SEK, which is equivalent to 8,823,637.78 SEK in January 2018. The award is presented in Stockholm at an annual ceremony on December 10, the anniversary of Nobel's death.[5]

In [45]:
for ent in test.ents:
    print(ent.text + ' -- ' + ent.label_ + ' -- ' + spacy.explain(ent.label_))

The Nobel Prize in Literature -- WORK_OF_ART -- Titles of books, songs, etc.
Swedish -- NORP -- Nationalities or religious or political groups
annually -- DATE -- Absolute or relative dates or periods
the Swedish Academy -- ORG -- Companies, agencies, institutions, etc.
one -- CARDINAL -- Numerals that do not fall under another type
Nobel Prizes -- WORK_OF_ART -- Titles of books, songs, etc.
1895 -- DATE -- Absolute or relative dates or periods
Alfred Nobel -- PERSON -- People, including fictional
Nobel's -- WORK_OF_ART -- Titles of books, songs, etc.
the Nobel Foundation -- ORG -- Companies, agencies, institutions, etc.
five -- CARDINAL -- Numerals that do not fall under another type
Swedish -- NORP -- Nationalities or religious or political groups
first -- ORDINAL -- "first", "second", etc.
Nobel Prize in Literature -- WORK_OF_ART -- Titles of books, songs, etc.
1901 -- DATE -- Absolute or relative dates or periods
1901 -- DATE -- Absolute or relative dates or periods
Prudhomme -- OR

In [31]:
dft_eng['annotations'][2]

"{'answer_start': array([129]), 'answer_text': array(['discourse between two or more people holding different points of view about a subject but wishing to establish the truth through reasoned arguments'],\n      dtype=object)}"

In [18]:
import re

print(dft_eng['answer_text'][4])
print(dft_eng['answer_text'][4].strip(']\'\'[').split(' '))
# print(re.split(r' |,|.', dft_eng['answer_text'][4].strip(']\'\'[')))

KeyError: 'answer_text'

In [85]:
dft_eng['answer_text_tokenized'] = [i.strip(']\'\'[').split(' ') for i in dft_eng.answer_text.values]

In [86]:
dft_eng['answer_text_tokenized']

0                                                 [1920s]
1                                      [Sully, Prudhomme]
2       [discourse, between, two, or, more, people, ho...
3                                    [Sejong, the, Great]
4       [Grasshoppers, are, plant-eaters,, with, a, fe...
                              ...                        
7384                                                   []
7385                                                   []
7386                                                   []
7387                                                   []
7388                                                   []
Name: answer_text_tokenized, Length: 7389, dtype: object

In [100]:
# A = dft_eng['document_plaintext_tokenized'][0]
# B = dft_eng['answer_text_tokenized'][0]
# IOB = []
# for i,x in enumerate(A):
#     if x in B and (i-1) >= 0 and IOB[i-1]=='O' or x in B and i==0:
#         IOB.append('B')
#     elif x in B and (i-1) >= 0 and IOB[i-1]=='B' or x in B and (i-1) >= 0 and IOB[i-1]=='I':
#         IOB.append('I')
#     else:
#         IOB.append('O')

# IOB

In [97]:
IOB = [[] for _ in range(dft_eng.shape[0])]
A = dft_eng['document_plaintext_tokenized']
B = dft_eng['answer_text_tokenized']

for i, elm in enumerate(A):
    for j, x in enumerate(elm):
        if x in B[i] and (j-1) >= 0 and IOB[i][j-1]=='O' or x in B[i] and j==0:
            IOB[i].append('B')
        elif x in B[i] and (j-1) >= 0 and IOB[i][j-1]=='B' or x in B[i] and (j-1) >= 0 and IOB[i][j-1]=='I':
            IOB[i].append('I')
        else:
            IOB[i].append('O')

dft_eng['IOB'] = IOB

In [102]:
dft_eng['answer_text'][4]

"['Grasshoppers are plant-eaters, with a few species at times becoming serious pests of cereals, vegetables and pasture, especially when they swarm in their millions as locusts and destroy crops over wide areas']"

In [101]:
dft_eng['IOB'][4]

['B',
 'I',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'B',
 'I',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O']

In [104]:
dft_eng['annotations'][4]

"{'answer_start': array([0]), 'answer_text': array(['Grasshoppers are plant-eaters, with a few species at times becoming serious pests of cereals, vegetables and pasture, especially when they swarm in their millions as locusts and destroy crops over wide areas'],\n      dtype=object)}"

#### (b)

In [115]:
dft_eng.iloc[4]

question_text                                           What do Grasshoppers eat?
document_title                                                        Grasshopper
language                                                                  english
annotations                     {'answer_start': array([0]), 'answer_text': ar...
document_plaintext              Grasshoppers are plant-eaters, with a few spec...
document_url                            https://en.wikipedia.org/wiki/Grasshopper
answer_start                                                                  [0]
answer_text                     ['Grasshoppers are plant-eaters, with a few sp...
question_text_tokenized                          [What, do, Grasshoppers, eat, ?]
document_plaintext_tokenized    [Grasshoppers, are, plant, -, eaters, ,, with,...
answer_text_tokenized           [Grasshoppers, are, plant-eaters,, with, a, fe...
labels                                                                          1
IOB             

In [111]:
import tensorflow as tf

new_df_val = tf.data.Dataset.from_tensor_slices(dict(dft_eng[['document_plaintext_tokenized', 'IOB']]))
new_df_val

2022-10-13 16:48:02.651412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [105]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
!unzip wiki-news-300d-1M.vec.zip

--2022-10-13 16:26:21--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650,22M  15,3MB/s    in 45s     

2022-10-13 16:27:07 (14,5 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]

Archive:  wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [106]:
# Reduce down to our vocabulary and word embeddings
def load_vectors(fname, vocabulary):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    tag_names = dft_eng.features[f"IOB"].feature.names
    final_vocab = tag_names + ['[PAD]', '[UNK]', '[BOS]', '[EOS]']
    final_vectors = [np.random.normal(size=(300,)) for _ in range(len(final_vocab))]
    for j,line in enumerate(fin):
        tokens = line.rstrip().split(' ')
        if tokens[0] in vocabulary or len(final_vocab) < 30000:
          final_vocab.append(tokens[0])
          final_vectors.append(np.array(list(map(float, tokens[1:]))))
    return final_vocab, np.vstack(final_vectors)

class FasttextTokenizer:
    def __init__(self, vocabulary):
        self.vocab = {}
        for j,l in enumerate(vocabulary):
            self.vocab[l.strip()] = j

    def encode(self, text):
        # Text is assumed to be tokenized
        return [self.vocab[t] if t in self.vocab else self.vocab['[UNK]'] for t in text]

In [109]:
vocabulary = (set([t for s in dft_eng for t in s['document_plaintext_tokenized']]) | set([t for s in dfv_eng for t in s['document_plaintext_tokenized']]))
vocabulary, pretrained_embeddings = load_vectors('wiki-news-300d-1M.vec', vocabulary)
print('size of vocabulary: ', len(vocabulary))
tokenizer = FasttextTokenizer(vocabulary)

TypeError: string indices must be integers

#### (c)

#### (d)

#### (e)